# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD model for fitting the actual Chilean data in order to make projections in VMI and bed usage, amount of infected and deaths. This model uses the SEIRHVD 4.1 version. A visual representation of the model is available at: [SEIRHVD Miró Model](https://miro.com/app/board/o9J_ktzN4xA=/)

This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from datetime import datetime
from numpy import linalg as LA
import multiprocessing
from joblib import Parallel, delayed
from datetime import timedelta

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD5 import SEIRHVD 
from Quarantine import Quarantine
from Quarantine import SeroPrevDynamics
from importdata import ImportData

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [2]:
# First simulation Initial date
initdate = datetime(2020,5,15)
# Date for change in SeroPrevalence
SPchange_date = datetime(2020,7,25)

# Current date
currentdate = datetime.now()
currentday = (currentdate - initdate).days

fiestaspatrias = datetime(2020,9,18)
fiestaspatrias_day = (fiestaspatrias-initdate).days
plebiscito = datetime(2020,10,25)
plebiscito_day = (plebiscito-initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. state region is represented by cut = 13.

In [3]:
tstate = '12'
# Import Data
state = ImportData(tstate=tstate,initdate = initdate)
state.importdata()

Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing Sochimi Data 2
Importing Accumulated Deaths
Importing Active Infected by Minciencia
Importing Hospitalized/NonHospitalized Deaths
Done


In [4]:
state.name = 'Región de Magallanes'
state.population

178362

## Datos de Hospitalizados a fitear

In [5]:
state.Hr = np.array(state.UTI)
state.Hr_tot = np.array(state.UTI_tot)

## Simulation Parameters

In [6]:
# Total simulation time
tsim = 1000
t_sp = (SPchange_date - initdate).days

beta = 0.074
mu = 0.3
k_I = 0
k_R = 0

SeroPrevFactor =0.2

expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [7]:
#quarantine = datetime(2020,8,22)
quarantine = datetime(2020,7,25)
qday = (quarantine-initdate).days
# Quarantines 
max_mob = 0.6
rem_mob = 0.35

Q1 = Quarantine(rem_mob,max_mob,qp=0,iqt=0,fqt=qday)
Q2 = Quarantine(rem_mob)

alpha = Q1.alpha

## Underreport
Fraction of Infected detected/reported  

$Imi_{det}$: Fraction of Mild detected   
$Ias_{det}$: fraction of asymptomatic detected  

If both are 1, means that all infected are detected so 
\begin{align}
I_{det} = I
\end{align}

In [8]:
Imi_det = 0.5
Ias_det = 0.3
Ise_det = 1
Icr_det = 1

## State Parameters:
The following parameters determine transitions probability and duration between different states. The values presented here are the default values set in the model. You can change them as it's showed in the following executable lines.

pE_Ias = 0.4  # Transition from exposed to Asymptomatic Infected  
tE_Ias = 5.0

pE_Imi = 0.55 # Transition from exposed to  Mild Infected  
tE_Imi = 5.0

pE_Icr = 0.01666 # Transition from exposed to  Critical Infected  
tE_Icr = 3.0

pE_Ise = 0.03334 # Transition from exposed to  Serious Infected  
tE_Ise = 3.0

pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered  
tIas_R = 10.0

pImi_R = 1.0  # Transition from Mild Infected to Recovered  
tImi_R = 15.0

pIse_Hse = 1.0 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)  
tIse_Hse = 3.0 

pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)  
tIse_D = 3.0         

pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)  
tIcr_V = 3.0 

pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)  
tIcr_D = 3.0         

pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered  
tHse_R = 11.0

pHse_V = 0.03 # Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)  
tHse_V = 3.0

pHse_D = 0.03 # Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)  
tHse_D = 3.0        

pV_Hout = 0.5  # Transition from Ventilators to Hospital Recovery (Hout)   
tV_Hout = 15.0

pV_D = 0.5 # Transition from Ventilators to Death  
tV_D = 15.0

pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered  
tHout_R = 4.0

pD_B = 1.0 # Transition from Dead to buried  
tD_B = 1.0 

betaD = 0 # Contagion by deads rate  
eta = 0.0 # Immunity loss rate  

In [9]:
#state parameters: 
pE_Ias = 0.433 # Transition from exposed to Asymptomatic Infected
tE_Ias = 4.0
pE_Imi = 0.55 # Transition from exposed to  Mild Infected
tE_Imi = 4.0
pE_Icr = 0.005 # Transition from exposed to  Critical Infected
tE_Icr = 3.0
pE_Ise = 0.002# Transition from exposed to  Serious Infected
tE_Ise = 3.0


pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered
tIas_R = 9.0
pImi_R = 1.0  # Transition from Mild Infected to Recovered
tImi_R = 12.0
pIse_Hse = 1 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)
tIse_Hse = 5.0
pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)
tIse_D = 3.0
pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)
tIcr_V = 3.0
pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)
tIcr_D = 3.0
pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered
tHse_R = 3.5
pHse_V = 0.03# Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)
tHse_V = 4.0
pHse_D = 0.85# Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)
tHse_D = 4.0
pV_Hout = 0.45  # Transition from Ventilators to Hospital Recovery (Hout)
tV_Hout = 16.0
pV_D = 0.55 # Transition from Ventilators to Death
tV_D = 12.0
pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered
tHout_R = 10.0

## SeroPrevalence Dynamics
The susceptibles increase through a step function which last for the specified amount of days. Throughout these days it adds a "dailyincrease" amount of persons each day.

The total increase of people is increasedays*dailyincrease

### First Susceptibles increase 2020-08-24

In [10]:
def chisum(a,b):
    def aux(t):
        return a(t)+b(t)
    return aux

In [11]:
# First increase
initincrease = datetime(2020,8,8)
initincreaseday = (initincrease - initdate).days
endincrease = datetime(2020,8,28)
increasedays = (endincrease - initincrease).days 
renewalFactor = 0.6
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays
t_sp_temp = t_sp + 0
chi0 = SeroPrevDynamics(initincreaseday,initincreaseday+increasedays*0.01,initincreaseday+increasedays,dailyincrease,form='quadratic')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)
chi = [chi0]
#renewalFactor = [renewalFactor]

In [12]:
renewalFactor = np.array([0.0,0.3,0.45,0.6,0.8])/1.5

In [13]:
# Increase date 
increasedate = datetime(2020,10,25)
increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 200
# Daily amount of people
dailyincrease = [state.population*SeroPrevFactor*i/increasedays for i in renewalFactor]

In [14]:
chi = [chisum(chi0,SeroPrevDynamics(increaseday-2,increaseday+increasedays*0.50001,increaseday+increasedays,i)) for i in dailyincrease]

In [15]:
chiplot = [[chi[i](t) for t in np.arange(0,tsim,0.1)] for i in range(len(renewalFactor))]
plt.title('Sero Prevalence Dynamics')
for i in range(len(chiplot)):
    plt.plot(np.arange(0,tsim,0.1),chiplot[i],label='RenewalFactor: '+str(renewalFactor[i]))
plt.axvline(x = t_sp, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
plt.axvline(x = fiestaspatrias_day, linestyle = 'dotted',color = 'red',label='18 Sept')
plt.axvline(x = plebiscito_day, linestyle = 'dotted',color = 'blue',label='Plebiscito')    
plt.xlim(0,200)
plt.legend(loc=0)

# Simulation

## Initial SeroPrev period
Initialize Simulation Object, set params and run the simulation. 

In [16]:
state.I_ac_r_dates[0]

datetime.datetime(2020, 5, 15, 0, 0)

In [17]:
simulation = [SEIRHVD(tsim,beta,mu,alpha,k_I=k_I,k_R = k_R, chi = i, SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,RealIC = state,Imi_det = Imi_det,Ias_det = Ias_det) for i in chi]

InitialCondition Object Data
InitialCondition Object Data
InitialCondition Object Data
InitialCondition Object Data
InitialCondition Object Data


Modify simulation state parameters

In [18]:
for i in range(len(simulation)):
    simulation[i].pE_Ias=pE_Ias
    simulation[i].tE_Ias=tE_Ias
    simulation[i].pE_Imi=pE_Imi
    simulation[i].tE_Imi=tE_Imi
    simulation[i].pE_Icr=pE_Icr
    simulation[i].tE_Icr=tE_Icr
    simulation[i].pE_Ise=pE_Ise
    simulation[i].tE_Ise=tE_Ise
    simulation[i].pIas_R=pIas_R
    simulation[i].tIas_R =tIas_R
    simulation[i].pImi_R=pImi_R
    simulation[i].tImi_R =tImi_R
    simulation[i].pIse_Hse=pIse_Hse
    simulation[i].tIse_Hse=tIse_Hse
    simulation[i].pIse_D=pIse_D
    simulation[i].tIse_D=tIse_D
    simulation[i].pIcr_V=pIcr_V
    simulation[i].tIcr_V=tIcr_V
    simulation[i].pIcr_D=pIcr_D
    simulation[i].tIcr_D=tIcr_D
    simulation[i].pHse_R=pHse_R
    simulation[i].tHse_R =tHse_R
    simulation[i].pHse_V=pHse_V
    simulation[i].tHse_V=tHse_V
    simulation[i].pHse_D=pHse_D
    simulation[i].tHse_D=tHse_D
    simulation[i].pV_Hout=pV_Hout
    simulation[i].tV_Hout =tV_Hout
    simulation[i].pV_D=pV_D
    simulation[i].tV_D =tV_D
    simulation[i].pHout_R=pHout_R
    simulation[i].tHout_R=tHout_R
    simulation[i].setnewparams()

Compartimental model State parameters changed
Compartimental model State parameters changed
Compartimental model State parameters changed
Compartimental model State parameters changed
Compartimental model State parameters changed


### Setting new params

## Run simulation

* **integr_sci**: Fast solver, sometimes stiffness beat it  
* **integr:** Slow solver, more robust with stiffness  

In [19]:
def simulate(simulation,i,tsim):
    simulation[i].integr_sci(0,tsim,0.1)
    return simulation[i]

In [20]:
num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(simulation,i,tsim) for i in range(len(simulation)))
simulation = sims
print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.0s
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    3.0s remaining:    4.5s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    3.0s remaining:    2.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    3.2s finished
ready


# Analysis

## Grid Plot

In [21]:
# Days to plot
days = 250
t_end = t_sp


enddate = initdate + timedelta(days=1.5*365)
#enddate = datetime(2020,12,31)

In [22]:
dates = [[initdate+timedelta(days=i) for i in simulation[j].t] for j in range(len(simulation))]

In [23]:
SeroPrevalence = [simulation[i].I_ac[-1]/simulation[i].population for i in range(len(simulation))]
SeroPrevalenceDet = [simulation[i].I_ac_det[-1]/simulation[i].population for i in range(len(simulation))]

In [24]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(renewalFactor)))

fig, axs = plt.subplots(2, 2)

for i in range(len(simulation)):
    axs[0,0].plot(dates[i],simulation[i].I_ac_det,label='RF: '+str(round(renewalFactor[i],2))+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
axs[0,0].set_title('Accumulated Infected')
axs[0,0].set_xlim(initdate,enddate)
axs[0,0].scatter(state.I_ac_r_dates,state.I_ac_r)
#axs[0,0].axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
axs[0,0].axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
axs[0,0].axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')

axs[0,0].axvline(x = datetime(2020,7,19), linestyle = 'dotted',color = 'grey',label='Paso a fase 3')
axs[0,0].axvline(x = datetime(2020,8,9), linestyle = 'dashed',color = 'grey',label='Paso a fase 2')
axs[0,0].axvline(x = datetime(2020,8,22), linestyle = 'dashdot',color = 'grey',label='Paso a fase 1')


axs[0,0].plot([], [], ' ', label='RF: RenewalFactor')
axs[0,0].plot([], [], ' ', label='SP_R: SeroPrevalence Real')
axs[0,0].plot([], [], ' ', label='SP_D: SeroPrevalence Detected')

axs[0,0].fmt_xdata = mdates.DateFormatter('%Y-%m-%d') 
axs[0,0].legend(loc=0)


for i in range(len(simulation)):
    axs[0,1].plot(dates[i],simulation[i].B,label='RenewalFactor: '+str(round(renewalFactor[i],2)),color=colors[i])
axs[0,1].set_title('Accumulated Deaths')
axs[0,1].set_xlim(initdate,enddate)
axs[0,1].scatter(state.Br_dates,state.Br,label='Muertes Totales')
#axs[0,1].scatter(state.hosp_dates,state.Br_hosp,label='Muertes Hospitalizados')
#axs[0,1].scatter(state.hosp_dates,state.Br_Nonhosp,label='Muertes No Hospitalizados')
#axs[0,1].scatter(state.hosp_dates,state.Br_Nonhosp,color = 'red',label='Muertes No Hospitalizados')
axs[0,1].axvline(x = datetime(2020,7,19), linestyle = 'dotted',color = 'grey',label='Paso a fase 3')
axs[0,1].axvline(x = datetime(2020,8,9), linestyle = 'dashed',color = 'grey',label='Paso a fase 2')
axs[0,1].axvline(x = datetime(2020,8,22), linestyle = 'dashdot',color = 'grey',label='Paso a fase 1')

#axs[0,1].axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
axs[0,1].axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
axs[0,1].axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')
axs[0,1].fmt_xdata = mdates.DateFormatter('%Y-%m-%d') 
axs[0,1].legend(loc=0)


#for i in range(len(simulation)):    
#    axs[1,0].plot(dates[i],simulation[i].Hse+simulation[i].Hout,label='RenewalFactor: '+str(renewalFactor[i]),color=colors[i])
#axs[1,0].set_title('UTI Usage')
#axs[1,0].set_xlim(initdate,datetime(2020,12,31))
#axs[1,0].scatter(state.sochimi_dates,state.UTI)
#axs[1,0].plot(dates[0],simulation[0].H_cap,label='UTI Capacity', linestyle = 'dashed', color = 'grey')
#axs[1,0].axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
#axs[1,0].axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
#axs[1,0].axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')
#axs[1,0].legend(loc=0)

for i in range(len(simulation)):    
    axs[1,0].plot(dates[i],simulation[i].I_d_det,label='RenewalFactor: '+str(round(renewalFactor[i],2)),color=colors[i])
axs[1,0].set_title('New Daily Infected')
axs[1,0].set_xlim(initdate,enddate)
axs[1,0].scatter(state.I_d_r_dates,state.I_d_r)

#axs[1,0].axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
axs[1,0].axvline(x = datetime(2020,7,19), linestyle = 'dotted',color = 'grey',label='Paso a fase 3')
axs[1,0].axvline(x = datetime(2020,8,9), linestyle = 'dashed',color = 'grey',label='Paso a fase 2')
axs[1,0].axvline(x = datetime(2020,8,22), linestyle = 'dashdot',color = 'grey',label='Paso a fase 1')

axs[1,0].axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
axs[1,0].axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')
axs[1,0].fmt_xdata = mdates.DateFormatter('%Y-%m-%d') 
axs[1,0].legend(loc=0)


for i in range(len(simulation)):        
    axs[1,1].plot(dates[i],simulation[i].V,label='RenewalFactor: '+str(round(renewalFactor[i],2)),color=colors[i])
    axs[1,1].plot(dates[i],simulation[i].V_need,linestyle='dashed',color=colors[i])
axs[1,1].set_title('VMI Usage')
axs[1,1].set_xlim(initdate,enddate)
axs[1,1].scatter(state.sochimi_dates,state.Vr)
axs[1,1].plot(dates[0],simulation[0].V_cap,label='VMI Capacity', linestyle = 'dashed', color = 'grey')
#axs[1,1].axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
axs[1,1].axvline(x = datetime(2020,7,19), linestyle = 'dotted',color = 'grey',label='Paso a fase 3')
axs[1,1].axvline(x = datetime(2020,8,9), linestyle = 'dashed',color = 'grey',label='Paso a fase 2')
axs[1,1].axvline(x = datetime(2020,8,22), linestyle = 'dashdot',color = 'grey',label='Paso a fase 1')

axs[1,1].axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
axs[1,1].axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')
axs[1,1].fmt_xdata = mdates.DateFormatter('%Y-%m-%d') 
axs[1,1].legend(loc=0)


fig.suptitle(state.name,fontsize=16)


Text(0.5, 0.98, 'Región de Magallanes')

In [672]:
type(simulation[0].I)

numpy.ndarray

In [673]:
SeroPrevalenceDet

[0.1368385493965184,
 0.1531416186379188,
 0.16129183185505935,
 0.1694434429178627,
 0.18031078315804094]

# Lockdown Data

In [27]:
endpoint = '../Data/Confinamiento_COVID19.xlsx'
Data = pd.read_excel(endpoint,header=1,index_col=0)
PA = Data['12101'].iloc[2:]

In [31]:
fig, ax = plt.subplots()
ax.plot(PA.astype(int))   
ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d') 
fig.suptitle(state.name+' Lockdown',fontsize=16)
plt.show() 

# Cálculo de Peak 

In [ ]:
idx = [np.where(np.array(dates[i])>datetime(2020,9,1))[0][0] for i in range(len(renewalFactor))] 

## Daily Infected

In [ ]:
for i in range(len(simulation)):
    plt.plot(dates[i],simulation[i].I_d,label='RenewalFactor: '+str(renewalFactor[i]),color=colors[i])
plt.axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
plt.axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
plt.axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')
plt.title('Daily new infected')
plt.legend(loc=0)
plt.show()

In [ ]:
peakdates_Id = [dates[i][np.where(simulation[i].I_d==max(simulation[i].I_d[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peakdates_Id

In [ ]:
peak_I_d = [simulation[i].I_d[np.where(simulation[i].I_d==max(simulation[i].I_d[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peak_I_d

In [ ]:
peak_I_d_VMI = [simulation[i].V[np.where(simulation[i].I_d==max(simulation[i].I_d[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peak_I_d_VMI

In [ ]:
peak_I_d_D = [simulation[i].D[np.where(simulation[i].I_d==max(simulation[i].I_d[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peak_I_d_D

## VMI

In [ ]:
VMIneed = [[simulation[i].V_d[j] + simulation[i].Icr_D_d[j] for j in range(len(simulation[i].V))]  for i in range(len(renewalFactor))]

In [ ]:
for i in range(len(simulation)):
    plt.plot(dates[i],VMIneed[i],label='RenewalFactor: '+str(renewalFactor[i]),color=colors[i])
plt.axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
plt.axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
plt.axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')
plt.title('New VMI daily need')
plt.legend(loc=0)
plt.show()

In [ ]:
peakdates_VMI = [dates[i][np.where(simulation[i].V==max(simulation[i].V[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peakdates_VMI

In [ ]:
peak_VMI = [simulation[i].V[np.where(simulation[i].V==max(simulation[i].V[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peak_VMI 

In [ ]:
peak_VMI_I_d = [simulation[i].I_d[np.where(simulation[i].V==max(simulation[i].V[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peak_VMI_I_d 

In [ ]:
peakdates_VMI_d = [dates[i][np.where(VMIneed[i]==max(VMIneed[i][idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peakdates_VMI_d

In [ ]:
peak_VMI_D = [simulation[i].D[np.where(simulation[i].V==max(simulation[i].V[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peak_VMI_D 

In [ ]:
state.Dr_hosp

## Peak Deaths


In [ ]:
peak_D = [simulation[i].D[np.where(simulation[i].D==max(simulation[i].D[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]
peak_D_dates = [dates[i][np.where(simulation[i].D==max(simulation[i].D[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]
peak_D_VMI = [simulation[i].V[np.where(simulation[i].D==max(simulation[i].D[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]
peak_D_I_d = [simulation[i].I_d[np.where(simulation[i].D==max(simulation[i].D[idx[i]:]))[0][0]] for i in range(len(renewalFactor))]

In [ ]:
peak_D

In [ ]:
peak_D_dates

In [ ]:
peak_D_VMI

In [ ]:
peak_D_I_d


In [ ]:
for i in range(len(simulation)):
    plt.plot(dates[i],simulation[i].D,label='RenewalFactor: '+str(renewalFactor[i]),color=colors[i])
plt.axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
plt.axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
plt.axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito')
plt.title('Daily Deaths')
plt.legend(loc=0)
plt.show()

## Single plots

### SEIRD

## Peak Date

In [ ]:
peakidx = np.where(state.Ir==max(state.Ir))[0][0]
datapeakdate = state.sochimi_dates[peakidx]

In [ ]:
datapeakdate

In [ ]:
simulation.peak_date

In [ ]:
datapeakdate

In [ ]:
Peak_delta = (simulation.peak_date -datapeakdate).days

In [ ]:
Peak_delta